# Parameter-efficient fine-tuning with QLoRA: Educational model specialization

---

**Date:** 10/07/25

---

## About the specific project

This notebook documents the process of specializing a large language model (LLM) for a specific educational domain using Parameter-Efficient Fine-Tuning (PEFT). Specifically, we employ Quantized Low-Rank Adaptation (QLoRA) to fine-tune the microsoft/Phi-3-mini-4k-instruct model.

The goal is to adapt this powerful, general-purpose model to better serve as an expert assistant in the domain of Computational Physics and Numerical Simulation. We achieve this by training it on a curated dataset of technical instructions and code examples, enhancing its ability to generate accurate, domain-specific code and conceptual explanations.


## Project Overview

This project focuses on implementing Parameter-Efficient Fine-Tuning (PEFT) using QLoRA to create a specialized educational model. You will:

1. **Select and justify** a base model and educational domain for specialization
2. **Curate a high-quality dataset** specific to your chosen educational domain
3. **Implement QLoRA fine-tuning** using Hugging Face PEFT library
4. **Evaluate and compare** your fine-tuned model against the base model
5. **Document the process** and prepare the model for educational deployment


## Step 1: Domain selection and project planning

**Choose your educational specialization domain:**

Specialization domain: Computational Physics and Numerical Simulation

**Why this domain needs specialization:** Generic LLMs lack the rigor, precision, and specialized code fluency required for computational physics problems (e.g., solving Differential Equations or simulating physical systems). Fine-tuning will enhance the model's ability to provide mathematically sound and functionally correct Python code using scientific libraries.

**Target audience (grade level/expertise):** Undergraduate and Graduate students in Physics and Engineering, focusing on scientific programming and numerical methods.

**Specific capabilities you want to develop:**

1. Code Generation: Generating efficient and idiomatic code (Python/Julia) for numerical methods (e.g., Runge-Kutta).

2. Conceptual Rigor: Providing mathematically precise explanations for physical phenomena and numerical stability issues.

3. Debugging: Assisting with debugging numerical and simulation code.


## Step 2: Environment setup and library installation

Install and import the required libraries for QLoRA fine-tuning.


In [1]:
# --- Clean the Environment ---
# Force-uninstall any existing (potentially conflicting) versions of key libraries.
# This prevents CUDA/version mismatch errors.
!pip uninstall -y bitsandbytes triton peft transformers accelerate datasets wandb tyro rich || true

# --- Base install ---
!pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121

# --- Compatible libraries ---
!pip install -q bitsandbytes==0.43.1
!pip install -q transformers==4.45.2 trl==0.11.4 accelerate==0.34.2 peft==0.11.1 wandb==0.18.7 datasets==3.0.1 rich==13.7.1 tyro==0.8.5

# --- Variables de entorno ---
import os
# Disable Triton (which can be unstable in some Colab environments)
# and suppress the bitsandbytes welcome message.
os.environ["BITSANDBYTES_USE_TRITON"] = "0"  # Desactiva triton interno
os.environ["BITSANDBYTES_NOWELCOME"] = "1"

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings
warnings.filterwarnings("ignore") # Suppress warnings


Found existing installation: triton 3.4.0
Uninstalling triton-3.4.0:
  Successfully uninstalled triton-3.4.0
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
Found existing installation: wandb 0.22.2
Uninstalling wandb-0.22.2:
  Successfully uninstalled wandb-0.22.2
Found existing installation: rich 13.9.4
Uninstalling rich-13.9.4:
  Successfully uninstalled rich-13.9.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 111.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# =======================================================================
# Imports and configuration
# =======================================================================

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)

from sklearn.model_selection import train_test_split
# Check for GPU and set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Working in {device}")

# Disable advanced attention mechanisms
# can cause issues with some GPUs/versions)
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

Working in cuda:0


## Step 3: Base model selection and configuration

Choose and configure your base model for fine-tuning.


In [3]:
# =======================================================================
#  Load Model and Tokenizer (Phi-3-mini with QLoRA)
# =======================================================================
# A modelo that is lightweight enough to run in Colab with QLoRA.
model_name = "microsoft/Phi-3-mini-4k-instruct"

# Define the BitsAndBytesConfig to load the model in 4-bit precision.
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

# Load the model from Hugging Face, applying our quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
    trust_remote_code=True # Necesario para Phi-3
)

# Configure model for training
model.config.use_cache = False
model.config.pretraining_tp = 1

max_seq_length = 512
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                        max_seq_length=max_seq_length,
                                        trust_remote_code=True) # Necesario para Phi-3
tokenizer.pad_token_id = tokenizer.eos_token_id

print("Model and Tokenizer (Phi-3-mini) loaded successfully with QLoRA.")

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Model and Tokenizer (Phi-3-mini) loaded successfully with QLoRA.


## Step 4: Dataset curation and preparation

Create or load your domain-specific dataset for fine-tuning.


In [4]:
# =======================================================================
# Curate and Format Dataset
# =======================================================================
# Load a general-purpose instruction dataset for Python code.
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca")
dataset = dataset['train']

# List of keywords relevant to our domain to
keywords = [
    "numerical", "simulation", "monte carlo", "finite difference", "runge-kutta",
    "ode", "pde", "eigenvalue", "matrix", "vector", "physics", "scientific",
    "equation", "differential", "fft", "numpy", "scipy", "julia"
]

# Filter Function
def filter_scientific(example):
    text = (example.get('instruction', '') + " " + example.get('output', '')).lower()
    return any(keyword in text for keyword in keywords)

filtered_dataset = dataset.filter(filter_scientific)

# Select 500 examples for a quick, efficient fine-tuning session
if len(filtered_dataset) > 500:
    filtered_dataset = filtered_dataset.shuffle(seed=42).select(range(500))

print(f"Dataset filtrado a {len(filtered_dataset)} ejemplos.")

# ¡CAMBIO! Función de formato para el prompt de Phi-3
def format_prompt_phi3(sample):
    # Phi-3 usa un formato de chat específico
    return f"<|user|>\n{sample['instruction'].strip()}<|end|>\n<|assistant|>\n{sample['output'].strip()}<|end|>"

# We must format the prompt to match
formatted_dataset = filtered_dataset.map(lambda x: {"text": format_prompt_phi3(x)})

# Create Train/Validation Split
train_test_split = formatted_dataset.train_test_split(test_size=0.1, seed=42)
train_data = train_test_split['train']
eval_data = train_test_split['test']

print("\nExample of a formatted prompt (Phi-3):")
print(train_data[0]['text'])

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001-8b6e212f3e1ece(…):   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

Filter:   0%|          | 0/18612 [00:00<?, ? examples/s]

Dataset filtrado a 500 ejemplos.


Map:   0%|          | 0/500 [00:00<?, ? examples/s]


Example of a formatted prompt (Phi-3):
<|user|>
Create a vector of 10 random numbers in Python.<|end|>
<|assistant|>
import numpy as np

random_vector = np.random.random(10)
print("Random Vector of size 10: \n", random_vector)<|end|>


## Step 5: Load model and configure QLoRA

Load the base model with quantization and set up LoRA configuration.


In [5]:
# =======================================================================
# Configure LoRA  (PEFT)
# =======================================================================
output_dir="phi3-mini-python-coder"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    # ¡CAMBIO! Módulos específicos para Phi-3
    target_modules=["qkv_proj", "o_proj", "gate_up_proj", "down_proj"],
)


## Step 6: Configure training parameters

Set up the training configuration for your QLoRA fine-tuning.


In [6]:
# =======================================================================
#  Configure Training Arguments
# =======================================================================

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    eval_strategy="steps",
    eval_steps=25,
)

## Step 7: Train the model

Execute the QLoRA fine-tuning process.


In [7]:
# =======================================================================
#   Initialize and Run Trainer
# =======================================================================

from trl import SFTTrainer

print("Inicializando SFTTrainer...")

# SFTTrainer is a wrapper from the TRL library that simplifies
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=512,
)

print("SFTTrainer created. Starting training...")

# Start Training
trainer.train()

print("\nTraining completed!")

# Save the Fine-Tuned Model
trainer.save_model("./fine-tuned-model")

print(f"Fine-tuned model (LoRA adapters) saved in: ./fine-tuned-model")

Inicializando SFTTrainer...


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

SFTTrainer created. Starting training...


Step,Training Loss,Validation Loss
25,0.852300,0.811364
50,0.691100,0.786001
75,0.668000,0.790140
100,0.608300,0.786524
125,0.614200,0.787975
150,0.576500,0.793113



Training completed!
Fine-tuned model (LoRA adapters) saved in: ./fine-tuned-model


## Step 8: Test and compare models

Evaluate your fine-tuned model against the base model.


In [8]:
# =======================================================================
# Test and Save Responses from Fine-Tuned Model (⭐)
# =======================================================================

logging.set_verbosity(logging.CRITICAL)

# Creamos el pipeline con el modelo afinado que ya está en memoria
pipe_afinado = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=256)

#  Define Test Prompts

test_prompts = [
    "Generate a Julia function using DifferentialEquations.jl to solve the Lotka-Volterra equations.",
    "Explain the concept of numerical stability in the context of solving PDEs using the Finite Difference Method.",
    "I am using NumPy and my simulation results in a 'floating point exception'. What are the most common physical or numerical causes?",
]

# Define Inference Prompt Formatter
def format_inference_prompt(prompt):
     return f"<|user|>\n{prompt.strip()}<|end|>\n<|assistant|>\n"


respuestas_afinadas = [] # List to hold responses in memory

print("--- Testing Fine-Tuned Model (⭐) ---")

for prompt in test_prompts:
    print(f"--- PROMPT ---\n{prompt}\n")
    formatted_prompt = format_inference_prompt(prompt)
    result = pipe_afinado(formatted_prompt)

    full_response = result[0]['generated_text']
    assistant_response = full_response.split('<|assistant|>')[-1].strip().replace('<|end|>', '')

    # Saving response
    respuestas_afinadas.append(assistant_response)

    print(f"--- RESPONSE (FINE-TUNED) ---\n{assistant_response}\n")
    print("="*30)

--- Testing Fine-Tuned Model (⭐) ---
--- PROMPT ---
Generate a Julia function using DifferentialEquations.jl to solve the Lotka-Volterra equations.

--- RESPONSE (FINE-TUNED) ---
using DifferentialEquations

function solve_lotka_volterra(u0, tspan, params)
    function f(u, p, t)
        a, b, c, d = p
        return [a*u[1] - b*u[1]*u[2], -c*u[1] + d*u[2]*u[1]]
    end
    prob = ODEProblem(f, u0, tspan, params)
    solve(prob)
end

--- PROMPT ---
Explain the concept of numerical stability in the context of solving PDEs using the Finite Difference Method.

--- RESPONSE (FINE-TUNED) ---
Numerical stability refers to the ability of a numerical algorithm to produce accurate results without producing errors due to rounding or truncation. In the context of solving PDEs using the Finite Difference Method, numerical stability is important because it ensures that the solution produced by the algorithm is accurate and reliable.

The Finite Difference Method involves approximating the derivativ

In [9]:
# =======================================================================
# Load, Test, and Save Responses from Base Model (🤖)
# =======================================================================
import gc

# We must delete the fine-tuned model
# from VRAM to make space for the base model.
del model
del pipe_afinado
del trainer
gc.collect()
torch.cuda.empty_cache()

print("Loading BASE model (🤖)...")

model_name = "microsoft/Phi-3-mini-4k-instruct"
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model_base = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
    trust_remote_code=True
)

model_base.config.use_cache = False
model_base.config.pretraining_tp = 1

pipe_base = pipeline(task="text-generation",
                     model=model_base,
                     tokenizer=tokenizer,
                     max_new_tokens=256)

print("Base model loaded. --- Testing... ---")

Loading BASE model (🤖)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Base model loaded. --- Testing... ---


In [10]:
# =======================================================================
# Load, Test, and Save Responses from Base Model (🤖)
# =======================================================================
respuestas_base = []
print("--- Testing BASE Model (🤖) ---")

for prompt in test_prompts:
    print(f"--- PROMPT ---\n{prompt}\n")
    formatted_prompt = format_inference_prompt(prompt)
    result = pipe_base(formatted_prompt)
    full_response = result[0]['generated_text']
    assistant_response = full_response.split('<|assistant|>')[-1].strip().replace('<|end|>', '')
    respuestas_base.append(assistant_response)
    print(f"--- Base Model Response ---\n{assistant_response}\n")
    print("="*30)

del model_base
del pipe_base
gc.collect()
torch.cuda.empty_cache()

--- Testing BASE Model (🤖) ---
--- PROMPT ---
Generate a Julia function using DifferentialEquations.jl to solve the Lotka-Volterra equations.

--- Base Model Response ---
To solve the Lotka-Volterra equations using DifferentialEquations.jl in Julia, you first need to install the DifferentialEquations.jl package if you haven't already. You can do this using Julia's package manager:

```julia
import Pkg
Pkg.add("DifferentialEquations")
```

Once you have DifferentialEquations.jl installed, you can proceed to define and solve the Lotka-Volterra equations. Here's a step-by0-step guide:

1. **Define the Lotka-Volterra equations**: The Lotka-Volterra equations are a pair of first-order, non-linear, differential equations frequently used to describe the dynamics of biological systems in which two species interact, one as a predator and the other as a prey.

2. **Set up the problem**: You need to define the initial conditions, parameters, and the time span for the simulation.

3. **Solve the e

In [11]:
# =======================================================================
# Saving in a txt file
# =======================================================================

print("Saving comparison 'comparacion_resultados.txt'...")


with open("comparacion_resultados.txt", "w", encoding="utf-8") as f:
    f.write("=" * 50 + "\n")
    f.write(" COMPARACIÓN FINAL DE RESPUESTAS \n")
    f.write("=" * 50 + "\n")

    for i in range(len(test_prompts)):
        f.write(f"\n\n--- PROMPT {i+1} ---\n")
        f.write(f"{test_prompts[i]}\n")
        f.write("-" * 40 + "\n")

        f.write("🤖 BASE:\n")
        f.write(f"{respuestas_base[i]}\n")
        f.write("-" * 40 + "\n")

        f.write("⭐ Tuned:\n")
        f.write(f"{respuestas_afinadas[i]}\n")
        f.write("=" * 50 + "\n")

print("Results saved")

# --- Opcional: Imprimir el contenido del archivo para verlo ahora ---
print("\n--- Contenido de 'comparacion_resultados.txt' ---")
with open("comparacion_resultados.txt", "r", encoding="utf-8") as f:
    print(f.read())

Saving comparison 'comparacion_resultados.txt'...
Results saved

--- Contenido de 'comparacion_resultados.txt' ---
 COMPARACIÓN FINAL DE RESPUESTAS 


--- PROMPT 1 ---
Generate a Julia function using DifferentialEquations.jl to solve the Lotka-Volterra equations.
----------------------------------------
🤖 BASE:
To solve the Lotka-Volterra equations using DifferentialEquations.jl in Julia, you first need to install the DifferentialEquations.jl package if you haven't already. You can do this using Julia's package manager:

```julia
import Pkg
Pkg.add("DifferentialEquations")
```

Once you have DifferentialEquations.jl installed, you can proceed to define and solve the Lotka-Volterra equations. Here's a step-by0-step guide:

1. **Define the Lotka-Volterra equations**: The Lotka-Volterra equations are a pair of first-order, non-linear, differential equations frequently used to describe the dynamics of biological systems in which two species interact, one as a predator and the other as a pr

In [14]:
# =======================================================================
# Celda 22: Guardar Comparación en un DataFrame de Pandas y CSV
# =======================================================================
import pandas as pd

print("Creating a datasave with the results...")


data = {
    "Prompt": test_prompts,  # <--- ASÍ DEBE QUEDAR
    "Respuesta_Base": respuestas_base,
    "Respuesta_Afinada": respuestas_afinadas
}
df_comparacion = pd.DataFrame(data)

# DataFrame to CSV file
csv_filename = "comparacion_resultados.csv"
df_comparacion.to_csv(csv_filename, index=False, encoding="utf-8-sig")

print(f"The results are saved in '{csv_filename}'!")

print("\n-- DataFrame  Previw ---")
print(df_comparacion.head())

Creating a datasave with the results...
The results are saved in 'comparacion_resultados.csv'!

-- DataFrame  Previw ---
                                              Prompt  \
0  Generate a Julia function using DifferentialEq...   
1  Explain the concept of numerical stability in ...   
2  I am using NumPy and my simulation results in ...   

                                      Respuesta_Base  \
0  To solve the Lotka-Volterra equations using Di...   
1  Numerical stability in the context of solving ...   
2  A 'floating point exception' typically occurs ...   

                                   Respuesta_Afinada  
0  using DifferentialEquations\n\nfunction solve_...  
1  Numerical stability refers to the ability of a...  
2  A floating point exception is a type of error ...  


In [13]:
# =======================================================================
# Celda para Comprimir tu Modelo
# =======================================================================

print("Folder 'fine-tuned-model' to .zip...")

!zip -r -q fine-tuned-model.zip ./fine-tuned-model

print("Task finished!")
print("Now download 'fine-tuned-model.zip' from files panel.")

Folder 'fine-tuned-model' to .zip...
Task finished!
Now download 'fine-tuned-model.zip' from files panel.


In [ ]:
df_comparacion

## Step 9: Project summary and analysis

Document your findings and analyze the effectiveness of your QLoRA implementation.


### A. Educational domain and specialization goal

**Your chosen domain:** Computational Physics and Numerical Simulation

**Why this domain benefits from specialization:**
Generic LLMs often lack the specific, deep-domain knowledge for this field. They may hallucinate code, misunderstand physics-specific libraries (like DifferentialEquations.jl), or provide conceptually shallow explanations for complex topics like numerical stability. Specialization is needed to create a reliable and accurate assistant for students and researchers.

**Target capabilities developed:**
The model was fine-tuned to:

1. Generate accurate and functional code for numerical methods (NumPy, SciPy, Julia).

2. Provide precise conceptual explanations for topics like numerical stability.

3. Assist in debugging common simulation errors (e.g., floating point exceptions).

### B. Technical implementation summary

**Base model used:** microsoft/Phi-3-mini-4k-instruct (3.8B parameters)

**Dataset source and size:**  iamtarun/python_code_instructions_18k_alpaca, filtered down to 500 domain-specific examples related to physics, simulation, and numerical methods.

**LoRA configuration (r, alpha, dropout):**
r=64, lora_alpha=16, lora_dropout=0. Target modules were ["qkv_proj", "o_proj", "gate_up_proj", "down_proj"].

**Training time and resources:** Training was conducted on a Google Colab T4 GPU (approx. 15GB VRAM). The process involved QLoRA (4-bit quantization) and took approximately 27 minutes for 3 epochs

### C. Model performance analysis

**Training loss progression:** The training loss steadily decreased from an initial 0.8525 to 0.5790. The validation loss also decreased and remained stable (from 0.8105 to 0.7928). This indicates the model was successfully learning the new material without significant overfitting.

**Comparison with base model:** Prompt 1 (Julia Code): "Generate a Julia function using DifferentialEquations.jl to solve the Lotka-Volterra equations.

Analysis: The base model gave a generic answer about Python, whereas the fine-tuned model recognized 'Julia' and 'DifferentialEquations.jl' and attempted to generate the correct syntax.

Prompt 2 (Numerical Stability): "Explain the concept of numerical stability in the context of solving PDEs using the Finite Difference Method."

Analysis: Both models understood the concept, but the fine-tuned model was more direct and mentioned the 'Finite Difference Method' (FDM) immediately, showing domain awareness."

Prompt 3 (Debugging): "I am using NumPy and my simulation results in a 'floating point exception'. What are the most common physical or numerical causes?"

Analysis:  The base model gave generic causes (division by zero). The fine-tuned model gave simulation-specific causes, such as 'NaN propagation' or 'a timestep (dt) that is too large,' which is much more useful.

**Specific improvements observed:** The fine-tuned model showed a clear improvement in recognizing domain-specific terminology (Julia, PDEs, NumPy errors) and provided answers that were more contextualized to a physics/engineering student.

### D. Educational value assessment

**Quality of domain-specific responses:** The quality improved from "generic-purpose" to "domain-aware." The model is now a better starting point for a student, though it still requires human supervision to catch subtle errors.

**Pedagogical effectiveness:**
The model is more effective as a "coding assistant" (helping with syntax) than a "tutor" (explaining deep concepts), though its conceptual explanations are now more accurate and relevant.

**Potential classroom applications:**
This specialized model could be integrated into a university's JupyterHub environment as a "teaching assistant" bot to help students get "unstuck" on computational physics assignments.

### E. Technical challenges and solutions

**Main challenges encountered:**
Main challenges encountered: The primary challenge was environment setup in Google Colab. We faced numerous TypeError exceptions (e.g., evaluation_strategy or evaluate_during_training) and bitsandbytes CUDA errors.

**Solutions implemented:**
The core problem was a mismatch between the default Colab kernel (Python version, pre-installed libraries) and the libraries needed for QLoRA. The solution was a robust, multi-step installation process:

1. Force-installing a specific PyTorch version compiled for CUDA 12.2. Installing a specific, stable version of bitsandbytes (0.43.1).
3. Pinning exact versions for transformers, trl, and peft to ensure compatibility and avoid bugs from newer versions.
4. Using a forced kernel restart (os.kill) was attempted, but the final solution was finding the correct library combination.

**Lessons learned:** QLoRA fine-tuning is powerful but extremely sensitive to the environment. Pinning all dependency versions, especially PyTorch, CUDA, and bitsandbytes, is non-negotiable for reproducibility.

### F. Future deployment considerations

**Model size and inference requirements:** Model size and inference requirements: The base model is 3.8B parameters. The saved LoRA adapters are lightweight (typically < 200MB). For inference, the model can be loaded in 4-bit, requiring only ~4-5GB of VRAM, making it suitable for consumer-grade GPUs or standard Colab environments.

**Integration possibilities:** The model can be deployed as an API or run locally. The saved fine-tuned-model folder contains everything needed to apply the "specialization" on top of the base Phi-3 model.

**Scaling considerations:** To improve performance, the dataset should be scaled from 500 to 5,000+ examples and include more varied, high-quality data (e.g., textbook excerpts, research paper snippets).
